Inspriation: [Quelle](https://nzlul.medium.com/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad)<br>
Daten: [Quelle](https://github.com/kitsamho/songlyrics_univeral_sentence_encoder/tree/master/Lyric_data)


## Imports

In [ ]:
#%conda install pip

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
from tensorflow.keras.utils import to_categorical


import nltk
from nltk.corpus import stopwords
import re

######## SEED setzen
import random
# Random Seed setzen:
random.seed(49)
np.random.seed(49)
SEED = 49


label_encoder = LabelEncoder()

## Load Data

In [ ]:
df = pd.read_csv('lyrics.csv', encoding='ISO-8859-1')
df.head(3)

In [ ]:
df.info()

Lyrics (Data) und Genere (Label) haben keinen Nullwert. Song (-Titel) fehelen 6 aber dass ist egal die können ignoriert werden.

In [ ]:
df.describe()

- **lyric_count**: Mit der split-Funktion wird die Lyric in ihre einzelnen Wörter (auch Kommas?) geteilt und dann mit len-Funktion angezeigt. Sprich die Anzahl an Wörtern im Songtext.
- **lyric_count_norm**: Dieses Feature entsteht, indem jeder Eintrag in der Spalte lyric_count mit dem max. aus lyric_count dividiert wird. So entsteht eine normalisierter Wert von lyric_count. Daraufhin wird dieser Wert der sich zwischen 1 und 0 befindet mit dem Wert 55 multipliziert. So entsteht ein neuer Wert in der Spalte lyric_count_norm. Der Wert wird sozusagen wieder hochskaliert.

- [Link zur Analyse der Daten](https://www.notion.so/Notizen-Ivan-10-04-2024-8ba4890572db452a91ded46a5a276060)

Meines erachten nichts relevantes ablesbar

## Utils

Funktion zum Plotten der Modell History

In [ ]:
# Helper function to plot the history of a model
def plot_history(history):
    print(history.history.keys())
    epochs = range(1, len(history.history['accuracy']) + 1)
    # summarize history for accuracy
    plt.plot(epochs, history.history['accuracy'], 'o-', label='Training accuracy')
    plt.plot(epochs, history.history['val_accuracy'], 'o-', label='Validation accuracy')
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(epochs)
    plt.legend(loc='upper left')
    plt.show()
    # summarize history for loss, start from second epoch, cause the first is always too high
    plt.plot(epochs[1:], history.history['loss'][1:], 'o-', label='Training loss')
    plt.plot(epochs[1:], history.history['val_loss'][1:], 'o-', label='Validation loss')
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(epochs[1:])  # Adjust x-axis to start from second epoch
    plt.legend(loc='upper left')
    plt.show()

## EDL und Preprocessing

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(df.genre)
plt.title('The distribution of genres')

Genres sehen erst Mal relativ gleichmäßig verteilt aus. Eventuell Später alle auf die ANzahl vom niedrigsten (Hip-Hop) kürzen 

Erstellen des numerischen Labels aus den Genres: 

In [ ]:
df['label'] = label_encoder.fit_transform(df['genre'])
df.head()

Preprocessing der Daten (lyrics):
- Lowercasing
- Entfernung von Zahlen oder Umwandlung in Text
- Entfernung von Satzzeichen
- Entfernung von Stoppwörtern
- Entfernung von Leerzeichen
- Entfernung von dopplungen: "yeah, yeah, yeah" -> "yeah"

In [ ]:
# Sicherstellen, dass NLTK-Ressourcen heruntergeladen sind
nltk.download('stopwords')

# Funktion zur Entfernung von Wiederholungen (z.B. "yeah yeah yeah" -> "yeah")
def remove_repetitions(text):
    # Aufteilen des Texts in Wörter
    words = text.split()
    # Erstellung eines neuen Texts ohne wiederholte Wörter
    new_text = ' '.join(sorted(set(words), key=words.index))
    return new_text

# Funktion zur Bereinigung des Lyrics-Textes
def clean_lyrics(text):
    # Lowercasing
    text = text.lower()
    # Entfernung von Zahlen
    text = re.sub(r'\d+', '', text)
    # Entfernung von Satzzeichen
    text = re.sub(r'[^\w\s]', '', text)
    # Entfernung von mehrfachen Leerzeichen
    text = re.sub(r'\s+', ' ', text)
    # Entfernung von Stoppwörtern
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Entfernung von Wiederholungen wie "yeah yeah yeah"
    text = remove_repetitions(text)
    return text

# Anwendung der Bereinigungsfunktion auf die Spalte 'lyrics'
df['lyrics'] = df['lyrics'].apply(clean_lyrics)


In [ ]:
df.head()

Train/Test-Split:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['lyrics'], df['label'], test_size=0.2, random_state=SEED, shuffle=True)

Tokenization:
The Tokenizer API from TensorFlow Keras can split sentences into words and encode them into integers.

The Tokenizer will perform all the necessary pre-processing steps:

- tokenize into word character (word level)
- num_words for maximum number of unique tokens
- filter out the punctuation terms
- convert all words to lower case
- convert all words to integer index

[Quelle](https://nzlul.medium.com/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad)


In [ ]:
# Defining pre-processing parameters
max_len = 182 # Das 2 Quartil genommen von oben lyrics_count
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # out of vocabulary token
vocab_size = 500

tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)

- num_words : how many unique word that we want to load in training and testing data
- oov_token : out of vocabulary token will be added to word index in the corpus which is used to build the model. This is used to replace out of vocabulary words (words that are not in our corpus) during text_to_sequence calls.

[Quelle](https://nzlul.medium.com/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad)


In [ ]:
# Get the word_index
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

The next step : Let’s represent each sentence by sequences of numbers using texts_to_sequencesfrom Tokenizer object. After that, we padded the sequence so that we can have same length of each sequence.
[Quelle](https://nzlul.medium.com/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad)


Traindata:

In [ ]:
training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)

Testdata:

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

- padding : ‘pre’ or ‘post (default pre). By using pre, we’ll pad before each sequence and post will pad after each sequence.
- maxlen : maximum length of all sequences. If not provided, by default it will use the maximum length of the longest sentence.
- truncating : ‘pre’ or ‘post’ (default ‘pre’). If a sequence length is larger than the provided maxlen value then, these values will be truncated to maxlen. ‘pre’ option will truncate at the beginning where as ‘post’ will truncate at the end of the sequences.

In [ ]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

## Modells

### Dense Model

Build Modell:

In [ ]:
# Define parameter
vocab_size = 500 
embedding_dim = 16
drop_value = 0.2
n_dense = 24

num_classes = df['label'].nunique() # Anzahl an Genres (für Softmax layer die Klassen)

# Define Dense Model Architecture
model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dim))
                    #,input_length = max_len''))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
model.summary()

Training des Modells:

In [ ]:
num_epochs = 1500
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# One-hot encode labels damit es mit es mit der loss-Function funktioniert muss hotencoded werden
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

history = model.fit(training_padded,
                    y_train_encoded,
                    epochs=num_epochs, 
                    validation_data=(testing_padded, y_test_encoded),
                    #callbacks =[early_stop],
                    verbose=2)

Evaluation:

In [ ]:
model.evaluate(testing_padded, y_test_encoded)

Plotting der History:

In [ ]:
plot_history(history)

In [ ]:
train_dense_results = model.evaluate(training_padded, np.asarray(y_train_encoded), verbose=2, batch_size=256)
valid_dense_results = model.evaluate(testing_padded, np.asarray(y_test_encoded), verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results[1]*100:0.2f}')
print(f'Valid accuracy: {valid_dense_results[1]*100:0.2f}')

TO-DO:
- Test und Validation muss getrennt werden.
- Modell muss optimiert werden

### LSTM Model

Build Modell:

In [ ]:
# Define parameter
n_lstm = 128
drop_lstm = 0.2
# Define LSTM Model 
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim))
model1.add(SpatialDropout1D(drop_lstm))
model1.add(LSTM(n_lstm, return_sequences=False))
model1.add(Dropout(drop_lstm))
model1.add(Dense(num_classes, activation='softmax'))

model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])


model1.summary()

Modell training:

In [ ]:
num_epochs1 = 30
early_stop1 = EarlyStopping(monitor='val_loss', patience=2)
history1 = model1.fit(training_padded,
                     y_train_encoded,
                     epochs=num_epochs1, 
                     validation_data=(testing_padded, y_test_encoded),
                     callbacks =[early_stop1],
                     verbose=2)

Plotting the results:

In [ ]:
plot_history(history1)

Print der Accuracy

In [ ]:
train_dense_results1 = model1.evaluate(training_padded, np.asarray(y_train_encoded), verbose=2, batch_size=256)
valid_dense_results1 = model1.evaluate(testing_padded, np.asarray(y_test_encoded), verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results1[1]*100:0.2f}')
print(f'Valid accuracy: {valid_dense_results1[1]*100:0.2f}')

TO-DO:
- Test und Validation muss getrennt werden.
- Modell muss optimiert werden

### Bi-LSTM Model

Modell Setup:

In [ ]:
model2 = Sequential()
model2.add(Embedding(vocab_size,
                     embedding_dim))
model2.add(Bidirectional(LSTM(n_lstm,
                              return_sequences = False)))
model2.add(Dropout(drop_lstm))
model2.add(Dense(num_classes, activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

model2.summary()

Modell Training:

In [ ]:
num_epochs2 = 30
early_stop2 = EarlyStopping(monitor = 'val_loss',
                           patience = 2)
history2 = model2.fit(training_padded,
                     y_train_encoded,
                     epochs = num_epochs2,
                     validation_data = (testing_padded, y_test_encoded),
                     callbacks = [early_stop2],
                     verbose = 2)

Plot History:

In [ ]:
plot_history(history2)

Print der Accuracy:

In [ ]:
train_dense_results2 = model2.evaluate(training_padded, np.asarray(y_train_encoded), verbose=2, batch_size=256)
valid_dense_results2 = model2.evaluate(testing_padded, np.asarray(y_test_encoded), verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results2[1]*100:0.2f}')
print(f'Valid accuracy: {valid_dense_results2[1]*100:0.2f}')

### Gated Recurrent Unit (GRU)

Modell Setup:

In [ ]:
model3 = Sequential()
model3.add(Embedding(vocab_size,
                     embedding_dim))
model3.add(SpatialDropout1D(0.2))
model3.add(GRU(128, return_sequences = False))
model3.add(Dropout(0.2))
model3.add(Dense(num_classes, activation='softmax'))
model3.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
model3.summary()

Modell Training:

In [ ]:
num_epochs3 = 30
early_stop3 = EarlyStopping(monitor='val_loss', patience=2)
history3 = model3.fit(training_padded,
                     y_train_encoded,
                     epochs=num_epochs3, 
                     validation_data=(testing_padded, y_test_encoded),
                     callbacks =[early_stop3],
                     verbose=2)

Plot History:

In [ ]:
plot_history(history3)

Print der Accuracy:

In [ ]:
train_dense_results3 = model3.evaluate(training_padded, np.asarray(y_train_encoded), verbose=2, batch_size=256)
valid_dense_results3 = model3.evaluate(testing_padded, np.asarray(y_test_encoded), verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results3[1]*100:0.2f}')
print(f'Valid accuracy: {valid_dense_results3[1]*100:0.2f}')

## Comparing Modells

In [ ]:
# Comparing the four different models
print(f"Dense model loss and accuracy: {model.evaluate(testing_padded, y_test_encoded)} \n------------------------------------------------------------\n" )
print(f"LSTM model loss and accuracy: {model1.evaluate(testing_padded, y_test_encoded)} \n------------------------------------------------------------\n" )
print(f"Bi-LSTM model loss and accuracy: {model2.evaluate(testing_padded, y_test_encoded)} \n------------------------------------------------------------\n" )
print(f"GRU model loss and accuracy: {model3.evaluate(testing_padded, y_test_encoded)}")

## Summary

Wir sehen aus dem vergleich der Modelle dass die modelle wie folgt abgeschnitten haben:
1. Platz: Dense
2. Platz: GRU
3. Platz: Bi-LSTM
4. Platz: LSTM

## Test with unseen data

TO-DO:
- Verschiedene Songs labeln lassen